In [1]:
import os
import cv2
import numpy as np
from PIL import Image,ImageDraw
import matplotlib.pyplot as plt 
%matplotlib inline

import tensorflow as tf
import tensorflow.keras.backend as ktf
from tensorflow.keras.models import load_model,Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model

from location.network import *
from recognition.network import *
from util import *
import cfg


## 模型载入

In [2]:
location_model = Location().location_network()
location_model.load_weights(cfg.location_weights)
location_model.summary()

#_, recognition_model = CRNN(cfg.width, cfg.height, cfg.label_len, cfg.characters).network()
#recognition_model.load_weights(cfg.recognition_weights)
#recognition_model.summary()


(None, 128, 128, 7)
Model: "Location"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 256, 256, 32) 896         input_img[0][0]                  
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 256, 256, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 256, 256, 32) 0           block1_conv1_bn[0][0]            
_______________________________________________________________________

## 可视化

In [ ]:
plot_model(location_model, to_file='EAST_Net.png')

In [ ]:
plot_model(recognition_model, to_file='CRNN_Net.png')

In [ ]:
def conv_output(model,layer_name,img):
    input_img = model.input
    try:
        # this is the placeholder for the conv output
        out_conv = model.get_layer(layer_name).output
    except:
        raise Exception('Not layer named {}!'.format(layer_name))

    # get the intermediate layer model
    intermediate_layer_model = Model(inputs=input_img, outputs=out_conv)

    # get the output of intermediate layer model
    intermediate_output = intermediate_layer_model.predict(img)
    
    featurema_map=intermediate_output[0]
    return featurema_map


In [ ]:
def vis_conv(images, n,t):
    """visualize conv output and conv filter.

    Args:
           img: original image.
           n: number of col and row.
           t: vis type.
           name: save name.
    """
    size = images.shape[0]
    margin = 0

    if t == 'filter':
        results = np.zeros((n * size + 7 * margin, n * size + 7 * margin, 3))
    if t == 'conv':
        results = np.zeros((n * size + 7 * margin, n * size + 7 * margin))

    for i in range(n):
        for j in range(n):
            if t == 'filter':
                filter_img = images[i + (j * n)]
            if t == 'conv':
                filter_img = images[..., i + (j * n)]
            filter_img = cv2.resize(filter_img, (size, size))

            # Put the result in the square `(i, j)` of the results grid
            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            if t == 'filter':
                results[horizontal_start: horizontal_end, vertical_start: vertical_end, :] = filter_img
            if t == 'conv':
                results[horizontal_start: horizontal_end, vertical_start: vertical_end] = filter_img

    # Display the results grid
    return results

In [ ]:
img = image.load_img('images/500.jpg')
d_wight, d_height = resize_image(img, cfg.image_size)
img = img.resize((d_wight, d_height), Image.NEAREST).convert('RGB')
plt.imshow(img)
img = image.img_to_array(img)
img = img/255*2-1
x = np.expand_dims(img, axis=0)

TypeError: add() takes 1 positional argument but 2 were given

In [ ]:
cout.shape

In [10]:
for index,layer in enumerate(location_model.layers):
    print(index,layer.name)

0 input_img
1 block1_conv1
2 block1_conv1_bn
3 block1_conv1_act
4 block1_conv2
5 block1_conv2_bn
6 block1_conv2_act
7 block2_sepconv1
8 block2_sepconv1_bn
9 block2_sepconv1_act
10 block2_sepconv2
11 block2_sepconv2_bn
12 conv2d
13 block2_sepconv2_act
14 batch_normalization
15 block2_pool
16 activation
17 add
18 block3_sepconv1
19 block3_sepconv1_bn
20 block3_sepconv1_act
21 block3_sepconv2
22 block3_sepconv2_bn
23 conv2d_1
24 block3_sepconv2_act
25 batch_normalization_1
26 block3_pool
27 activation_1
28 add_1
29 block4_sepconv1
30 block4_sepconv1_bn
31 block4_sepconv1_act
32 block4_sepconv2
33 block4_sepconv2_bn
34 conv2d_2
35 block4_sepconv2_act
36 batch_normalization_2
37 block4_pool
38 activation_2
39 add_2
40 up_sampling2d
41 concatenate
42 conv2d_3
43 batch_normalization_3
44 activation_3
45 conv2d_4
46 batch_normalization_4
47 activation_4
48 attention1
49 up_sampling2d_1
50 concatenate_1
51 conv2d_6
52 batch_normalization_5
53 activation_5
54 conv2d_7
55 batch_normalization_6
56

In [4]:
print(layer.output for layer in location_model.layers)

<generator object <genexpr> at 0x0000019FD9F9DC00>


In [ ]:
layer_name='block3_sepconv2'
#layer_name='block4_pool'
#layer_name='up_sampling2d_2'
#layer_name='conv2d_6'
#layer_name='concatenate_1'
#layer_name='att'
#layer_name='inside_score'
#layer_name='side_vertex_code'
#layer_name='side_vertex_coord'
cout=conv_output(location_model,layer_name,x)
feamap=vis_conv(cout,1,'conv')
plt.imshow(feamap)

In [ ]:
plt.figure(figsize=(5,5))
for i in range(2,5):
    layer_name='block'+str(i)+'_pool'
    cout=conv_output(location_model,layer_name,x)
    y=cout[:,:,9:12]
    
    plt.subplot(1,5,i+1)
    plt.title(layer_name)
    plt.xlabel('off')
    plt.imshow(y)

In [ ]:
n=3
for i in range(n):
    if i==0:
        layer_name='up_sampling2d'
    else:
        layer_name='up_sampling2d_'+str(i)
    y=conv_output(location_model,layer_name,x)[:,:,:]
    #plt.figure(dpi=50)
    #plt.figure(figsize=(10,10),dpi=100)
    #plt.figure(i+1,figsize=(10,10),dpi=600)
    plt.subplot(1,n,i+1)

    plt.xlabel(layer_name)
    plt.imshow(y)
    #plt.show()